In [1]:
suppressMessages(library(ArchR))
suppressMessages(library(patchwork))
suppressMessages(library(ggplot2))
suppressMessages(library(dplyr))
suppressMessages(library(cowplot))
suppressMessages(library(ggrepel))

In [2]:
set.seed(42)
addArchRThreads(threads = parallel::detectCores() - 2)
addArchRGenome("hg38")

Setting default number of Parallel threads to 126.

Setting default genome to Hg38.



In [3]:
proj <- loadArchRProject("../data/VisiumHeart", showLogo = FALSE)

Successfully loaded ArchRProject!



In [4]:
proj <- addGroupCoverages(ArchRProj = proj, groupBy = "Sample",
                          maxCells = 5000,
                          minCells = 5000,
                          force = TRUE)

ArchR logging to : ArchRLogs/ArchR-addGroupCoverages-3e0ae01fc58622-Date-2021-09-16_Time-11-10-03.log
If there is an issue, please report to github with logFile!

CK166 (1 of 8) : CellGroups N = 2

CK167 (2 of 8) : CellGroups N = 2

CK168 (3 of 8) : CellGroups N = 2

CK169 (4 of 8) : CellGroups N = 2

CK170 (5 of 8) : CellGroups N = 2

CK171 (6 of 8) : CellGroups N = 2

CK173 (7 of 8) : CellGroups N = 2

CK174 (8 of 8) : CellGroups N = 2

2021-09-16 11:10:08 : Creating Coverage Files!, 0.09 mins elapsed.

2021-09-16 11:10:08 : Batch Execution w/ safelapply!, 0.09 mins elapsed.

2021-09-16 11:10:58 : Adding Kmer Bias to Coverage Files!, 0.924 mins elapsed.

Completed Kmer Bias Calculation

Adding Kmer Bias (1 of 16)

Adding Kmer Bias (2 of 16)

Adding Kmer Bias (3 of 16)

Adding Kmer Bias (4 of 16)

Adding Kmer Bias (5 of 16)

Adding Kmer Bias (6 of 16)

Adding Kmer Bias (7 of 16)

Adding Kmer Bias (8 of 16)

Adding Kmer Bias (9 of 16)

Adding Kmer Bias (10 of 16)

Adding Kmer Bias (11 

In [5]:
## peak calling
pathToMacs2 <- findMacs2()

proj <- addReproduciblePeakSet(
    ArchRProj = proj, 
    groupBy = "Sample", 
    cutOff = 0.05,
    pathToMacs2 = pathToMacs2
)

proj <- addPeakMatrix(proj)

Searching For MACS2..

Found with $path!

ArchR logging to : ArchRLogs/ArchR-addReproduciblePeakSet-3e0ae011b57349-Date-2021-09-16_Time-12-07-05.log
If there is an issue, please report to github with logFile!

Calling Peaks with Macs2

2021-09-16 12:07:05 : Peak Calling Parameters!, 0.003 mins elapsed.



      Group nCells nCellsUsed nReplicates nMin nMax maxPeaks
CK166 CK166   3402        540           2   40  500   150000
CK167 CK167   2575        540           2   40  500   150000
CK168 CK168   2156        540           2   40  500   150000
CK169 CK169    856        540           2   40  500   150000
CK170 CK170   1651        540           2   40  500   150000
CK171 CK171   5033        540           2   40  500   150000
CK173 CK173   1743        540           2   40  500   150000
CK174 CK174    284        284           2   40  244   142000


2021-09-16 12:07:06 : Batching Peak Calls!, 0.003 mins elapsed.

2021-09-16 12:07:06 : Batch Execution w/ safelapply!, 0 mins elapsed.

2021-09-16 12:10:21 : Identifying Reproducible Peaks!, 3.258 mins elapsed.

2021-09-16 12:10:33 : Creating Union Peak Set!, 3.464 mins elapsed.

Converged after 6 iterations!

Plotting Ggplot!

2021-09-16 12:10:41 : Finished Creating Union Peak Set (223167)!, 3.595 mins elapsed.

ArchR logging to : ArchRLogs/ArchR-addPeakMatrix-3e0ae010664d98-Date-2021-09-16_Time-12-10-41.log
If there is an issue, please report to github with logFile!

2021-09-16 12:10:41 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-addPeakMatrix-3e0ae010664d98-Date-2021-09-16_Time-12-10-41.log



In [6]:
## save peak matrix
peakMatrix <- getMatrixFromProject(proj,
                                   useMatrix = "PeakMatrix")

counts <- peakMatrix@assays@data$PeakMatrix
df_rangers <- as.data.frame(peakMatrix@rowRanges@ranges)

rownames(counts) <- paste(peakMatrix@rowRanges@seqnames,
                          df_rangers$start,
                          df_rangers$end,
                          sep = "_") 

saveRDS(counts, file = "../data/VisiumHeart/PeakMatrix.Rds")

ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-3e0ae0294c2410-Date-2021-09-16_Time-12-14-07.log
If there is an issue, please report to github with logFile!

2021-09-16 12:14:40 : Organizing colData, 0.55 mins elapsed.

2021-09-16 12:14:40 : Organizing rowData, 0.552 mins elapsed.

2021-09-16 12:14:40 : Organizing rowRanges, 0.552 mins elapsed.

2021-09-16 12:14:41 : Organizing Assays (1 of 1), 0.552 mins elapsed.

2021-09-16 12:14:55 : Constructing SummarizedExperiment, 0.791 mins elapsed.

2021-09-16 12:14:56 : Finished Matrix Creation, 0.804 mins elapsed.



In [7]:
## save gene matrix
atac <- getMatrixFromProject(ArchRProj = proj,
                             useMatrix = "GeneScoreMatrix")

counts <- atac@assays@data$GeneScoreMatrix
rownames(counts) <- atac@elementMetadata$name

saveRDS(counts, file = "../data/VisiumHeart/GeneScoreMatrix.Rds")

ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-3e0ae04cfea0c5-Date-2021-09-16_Time-12-17-32.log
If there is an issue, please report to github with logFile!

2021-09-16 12:17:58 : Organizing colData, 0.434 mins elapsed.

2021-09-16 12:17:58 : Organizing rowData, 0.436 mins elapsed.

2021-09-16 12:17:58 : Organizing rowRanges, 0.436 mins elapsed.

2021-09-16 12:17:58 : Organizing Assays (1 of 1), 0.436 mins elapsed.

2021-09-16 12:18:14 : Constructing SummarizedExperiment, 0.709 mins elapsed.

2021-09-16 12:18:15 : Finished Matrix Creation, 0.724 mins elapsed.



In [ ]:
saveArchRProject(ArchRProj = proj, 
                 load = FALSE)
sessionInfo()